<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Diode 02 : Diode at large signal

This project deals with models used to obtain the response of a diode against large signals.

Version 1.2 (13/3/2019)  
License information is at the end of the document

---
**Bill Of Materials (BOM):**

* Diodes: **1N4148**
* Resistor: $1 k\Omega$

---

## Diode calculation problems

As we know, a semiconductor diode is a component that can be modeled by an exponential function:

$$I_d(V_d) = I_S \left( e^{V_d \; / \; \eta V_T} -1 \right)$$  

This model is a very good one for DC operation. You only need to know the value of the $I_S$ and $\eta$ parameters ($V_T$ is calculated from the diode temperature, about $26 mV$ at $300 K$) and it provides a good agreement with the real operation of the diode.

Unfortunately this is a difficult model to use for hand calculations. Let's try, for instance, to solve the following circuit:

![fig 01](images\D_02\fig01.png)

Using some circuit theory we get:

$$V_S - I_d R_1 - V_d = 0$$

Joining this equation with the diode equation we get a system:

> $\quad I_d(V_d) = I_S \left( e^{V_d \; / \; \eta V_T} -1 \right)$  
> $\quad V_S - I_d R_1 - V_d = 0$  

We can take out the $I_d$ unknown joining the two equations:

$$V_S - R_1 I_S \left( e^{V_d \; / \; \eta V_T} -1 \right) - V_d = 0$$

We just need to isolate $V_d$ to obtain the solution. Try that yourself.

You can't? Off course you can't. Nobody can.

You can solve the above equation numerically but you cannot isolate the $V_d$ variable. 

There is an easy numeric way, isolate the exponential:

$$e^{V_d \; / \; \eta V_T} = \frac{V_S-V_d}{R_1 I_S}+1$$

Then take logarithms on both sides and leave $V_d$ alone on one side:

$$V_d = \eta V_T ln \left( \frac{V_S-V_d}{R_1 I_S}+1 \right)$$

This is an interesting equation because it uses $V_d$ as its input parameter and gets $V_d$ also as its output. If we feed the equation with an initial seed Vd value and we input in the equation each solution we get, it converges to the desired solution for Vd.

Let's solve the equation for the **1N4148** diode and $Vs = 2.5 V$, $R1 = 1 k\Omega$.

Recall the $I_S$ and $\eta$ values measured previously for the diode and use the following code cell to solve the equation.

In [ ]:
# Import the math module
import math

# Write down the values obtaned in the previou notebook
Is = 10e-6 # Is value (mA)
n =  2 # n value (close to 2)
Vt = 0.026  # (V)

# Circuit parameters
Vs = 2.5  # (V)
R1 = 1    # (kOhm)

# Define the Vd function
func = lambda x: n*Vt*math.log(((Vs-x)/(R1*Is))+1)

# Set a seed value
Vd = 0.0

Each time you execute the following cell, you will perform an iteration of the $V_d$ equation. Execute the cell until the values are stable up to the $mV$ range.

In [ ]:
# Perform one iteration
Vd = func(Vd)
print('Vd =',Vd,'V')

---

![Practical Icon](images/pt.png)

---

Now, we want to check how the result compares with real measurements.

![fig 02](images\D_02\fig02.png)

First we will **import** SLab and **connect** to the board.

In [ ]:
# Import the SLab module
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

It is always a good idea to check the board calibration.

In [ ]:
# Check the calibration
slab.checkCalibration(pause=False,na=4)

---

**BUILD TASK** 
Mount the proposed circuit.   

---

Now we can measure the $V_d$ voltage and compare it with our calculations.

In [ ]:
# Set DAC1 to 2.5V
slab.setVoltage(1,2.5)

# Measure Vd voltage
Vd_meas = slab.readVoltage(1)
print('Vd =',Vd_meas,'V')

We see that we can solve the circuit, but it is not easy and we don't get a closed formula for the circuit variables.

Moreover, the numerical solution works because the solution converges, but if you had isolated the Vd variable in a different way like:

$$V_d = V_S - R_1 I_S \left( e^{V_d \; / \; \eta V_T} -1 \right)$$


It would not work as this equation diverges. Try it yourself:

In [ ]:
# Define the Vd function
func2 = lambda x: Vs-R1*Is*(math.exp(x/(n*Vt))-1)

# Set a seed value
Vd = 0.0

In [ ]:
# Iteration cell
Vd = func2(Vd)
print('Vd =',Vd,'V')

You could argue that we have only solved a two component circuit, but this method can be used always as long as the diode is the only non linear element in the circuit. You can always make the [Thevenin](https://en.wikipedia.org/wiki/Th%C3%A9venin%27s_theorem) equivalent of the circuit as seen from the diode to obtain a circuit like the one we have solved.

![fig 03](images\D_02\fig03.png)

There is no problem for computers to use numeric methods to obtain the diode solution, but for hand calculation it is cumbersome. Let's try a different way.

## Graphical solution

If we recall the circuit we are trying to solve, we know that it is defined by two equations:

> $\quad I_d(V_d) = I_S \left( e^{V_d \; / \; \eta V_T} -1 \right)$  
> $\quad V_S - I_d R_1 - V_d = 0$  

The first equation is the non linear diode equation and the second equation is a linear equation that represents the rest of the circuit. We can isolate $I_d$ on this second equation:

> $\quad I_d(V_d) = I_S \left( e^{V_d \; / \; \eta V_T} -1 \right)$  
> $\quad I_d = \frac{V_S-V_d}{R_1}$  

The solution to the system is the $V_d$ value that makes both equations provide the same $I_d$ value.
As the second equation is linear and non time dependent, it can be represented by a straight line.
In order to obtain this solution graphically we only need to draw this line on top of the diode curve.

To do that, we will first recall the diode curve we saved in the previous notebook. Then,  for our considered case we can obtain the solution of the linear part of the circuit and plot it together with the diode curve.

We will make plots interactive so that you can zoom on the graph.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

# Load the diode curve
# We assume it is in the standard SLab "Files" folder
vd,id = slab.load("../Files/1N4148")

# Obtain the curve of the linear part of the circuit
id2 = (2.5-vd)/1

# Plot both curves together
slab.plot1n(vd,[id,id2],"","Vd (V)","Id (mA)")

Zoom in on the point where the curve and the line cross to find the solution and compare it with the numeric and measured values.

The graphical method was a very useful method when we didn't have computers to do the numeric calculations as you only needed to draw a line over the diode curve provided by the manufacturer.

<div class="alert alert-block alert-warning">
<font size="5" color=brown>Graphical method in other components</font><font color=black> 
<BR><BR>
The method can be used with other non linear two terminal components, just replace the diode curve with the non linear component curve. For some components, however, there can be some problems with this method. In our case the diode curve and the line only crosses in one point that is the solution for the system. For some non linear components there can be more that one crossing point and you need to use other methods to determine which point is the good one.
<BR></font></div>

## Make it easy

The graphical method is a step forward to solving the diode equation, especially if you don't have a computer at hand, but it is not fast and, also, has an inherent problem shared with the numerical solution: it only works if there is only one diode in the circuit.

You can find numerical solution for more than one diode, but they get quite complicated. The best numerical solution for more than one diode is to use a circuit simulator like one of the [spice](https://es.wikipedia.org/wiki/SPICE) variants.

One way to simplify diode calculations is to use a simpler diode model. When you change the model you have a compromise: on one hand you ease the diode calculations, on the other hand you get less precise solutions.

![fig 04](images\D_02\fig04.jpg)

The constant voltage diode mode makes use of the fact that the diode voltage has a very small change of voltage for a broad current range. For instance, if we get the **1N4148** current as an example, as seen on the above figure, we can see that the diode voltage changes from $0.6 V$ to $0.7 V$ for current values from $0.7 mA$ to $5 mA$. So, we can suppose that the voltage is constant at a value of $0.65V$ as long as there is current in the diode. Most people, however, pick a value of $0.7 V$ for the diode mean forward voltage so that will be our chosen value.

We can define the average diode voltage when it conducts as $V_\gamma$ and define the diode operation depending in two possible states: ON and OFF. That gives us the constant voltage model equations:

> ON State $\quad V_d = V_\gamma \quad I_d \geq 0$  
> OFF State $\quad I_d = 0_\gamma \quad V_d \leq V_\gamma$  

The following figure shows the **1N4148** curve, in blue, together with the constant voltage model, in green, if we define $V_\gamma$ as $0,7V$.

![fig 05](images\D_02\fig05.png)

In order to use this model we need to know if the diode is in the **ON** or the **OFF** state. Unfortunately, as we have not yet solved the circuit, we don't know the state. To solve the problem we make a hypothesis, solve the circuit, and check if the hypothesis suits the model. If not, we know that the hypothesis is false and we need to use the other hypothesis. 

For the constant voltage diode model the hypothesis, equations and checks are:

![fig 06](images\D_02\fig06.png)

The method goes as follows:

>**(1)** Select one hypothesis

>**(2)** Solve the circuit using the diode equation for that hypothesis.
>>Solve also the variable needed to check the hypothesis.

>**(3)** Check the hypothesis. If it fails, return to **(1)** selecting another hypothesis.

As the constant voltage diode model has only two hypotheses, if the selected one is false, the other needs to be true. It is advisable, however, to always check the hypothesis to detect possible calculation errors.

We will solve the previously defined circuit wit $V_S = 2.5 V$ and $R_1 = 1 k\Omega$ using the constant voltage diode model with $V_\gamma = 0.7 V$.

---

**CALCULATION TASK**  
Solve the circuit using the constant voltage diode model.   
Try first to use the OFF hypothesis and change to the ON hypothesis if it fails.   
Which is the good hypothesis? What are the obtained values for $V_d$ and $I_d$?        

---

You should get a good agreement with the measurements, especially in the $I_d$ value. This model depends on having a circuit that does not depend on knowing the exact $V_d$ voltage. So, if we connect the diode to a linear circuit, the higher the **Thevenin** open circuit voltage is, the better the model will work.

<div class="alert alert-block alert-warning">
<font size="5" color=brown>Constant voltage diode model for more than one diode</font><font color=black> 
<BR><BR>
If you have more than one diode in the circuit, you can also use the constant voltage diode model. The hypotheses, however, are made for all the diodes at the same time. For two diodes, as each one has two possible states (ON, OFF) there are four possible hypotheses (OFF;OFF), (OFF, ON), (ON,OFF), (ON, ON). For three diodes there are nine hypotheses and so on.
<BR><BR>
If only one diode fails the check of one hypothesis you need to reject the hypothesis and you don't get information on the state of any diode. You cannot assume that the diodes that checked ok the hypothesis are in the correct state if any other diode fails the hypothesis.
<BR><BR>
In order to ease the problem it is good to apply some common sense to select the hypothesis to try. For instance, if several diodes are in series, the only possible hypotheses are all ON or all OFF.
<BR></font></div>

The constant voltage diode model, due to its simplicity, is the most used diode model for hand calculations. In general, if we need a more exact model we use a circuit simulation package.

## Last comments

This was a short project to introduce the constant voltage diode model that is usually used for hand calculations of circuits that include diodes.

We have seen also other methods to calculate circuits with diodes, like the graphical method or the numeric method but those methods are cumbersome and only work with only one diode in the circuit.

In general, for large signal DC calculations, we will use the constant voltage diode model if we need fast not too exact solutions and we will revert to circuit analysis packages for the case that the precision of the model is not enough.

## References

[SLab Python References](../Reference)  
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module. 
They should be available in the SLab/Doc folder.

[TinyCad](https://sourceforge.net/projects/tinycad/)  
Circuit images on this document have been drawn using the free software TinyCad  

[SciPy](https://www.scipy.org/)  
All the functions plots have been generated using the Matplotlib SciPy package. 

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">